# Symbulate Documentation: Random Variables

<a id='contents'></a>

1. [**Introduction and installing Symbulate**](index.html)
1. [**Probability spaces**](probspace.html)
1. [**Simulation tools**](sim.html)
1. [**Random variables**](#rv)
  1. [**Defining a random variable with RV:**](#RV)
  1. [**Plot:**](#plot) Use **.plot()** to display a plot summary of the simulated values of a RV.
  1. [**Distribution of a random variable:**](#distribution)
  1. [**Functions of a random variable:**](#transform)
  1. [**Mean:**](#mean) Use **.mean()** to compute the average of the simulated values of a RV.
  1. [**Variance:**](#var): Use **.var()** to compute the variance of the simulated values of a RV.
  1. [**Standard deviation:**](#var): Use **.sd()** to compute the standard deviation of the simulated values of a RV.
1. [**Commonly used probability models**](common.html)
1. [**Multiple random variables and joint distributions**](joint.html)
1. [**Conditioning**](conditioning.html)
1. [**Random processes**](process.html)
1. [**Markov processes**](mc.html)

Instructions for downloading and installing Symbulate can be found [here](http://calpoly.edu/~dsun09/python.html).  During a session, be sure to import Symbulate using the following commands.
<a id='rv'></a>

In [ ]:
from symbulate import *
%matplotlib inline

## Random variables

A random variable is a function that takes as an input an outcome of a probability space and returns a real number.  Commonly encountered random variables are either discrete or continuous.
  * A **discrete** random variable can take at most countably many distinct values.
  * A **continuous** random variable can take any value in some interval of real numbers.

Values of a random variable can be simulated and analyzed much like the outcomes of a probability space.  However, several additional tools are available for RV.

<a id='RV'></a>
[Back to contents](#contents)

### Defining a random variable with RV

Use the RV class to define random variables on a probability space.  A random variable is a function which maps the outcomes of a probability space to a real number; therefore a probability space must be defined first.

*Example.* Consider the random variable $X$ representing the number of Heads in 5 flips of a fair coin.  With 1 representing Heads and 0 Tails, we can count the number of Heads by summing the 0/1 values.  We define a probability space corresponding to 5 flips of a fair coin, and then a RV on that space which takes an outcome and sums its values.

In [ ]:
P = BoxModel([0,1], size = 5)
X = RV(P, sum)

If no function is specified in RV, the default is the identity function.

Like the probability space defintions, the RV command only defines a random variable.  [Simulation tools](sim.html) like `.sim()`, `.tabulate()`, `.filter()`, and `.count()`  work for `RV` just like for `ProbabilitySpace`.

In [ ]:
P = BoxModel([0,1], size = 5)
X = RV(P, sum)
values = X.sim(10000)
values

In [ ]:
values.tabulate(normalize = True)

In [ ]:
values.count_gt(3) / 10000

Random variables can also be specified by user defined functions.

*Example.* Ten cards labeled 1, 2, $\ldots$ , 10 are shuffled and dealt one at a time. Let $X$ be the number of the cards (out of ten) for which the number on the card matches its position in the deal. (For example, a match occurs if card 3 is the third card dealt.)

In [ ]:
n = 10
labels = list(range(n))
# remember, Python starts the index at 0, so the cards are labebeled 0, ..., 9

def number_matches(x):
    count = 0
    for i in range(n):
        if x[i] == labels[i]:
            count += 1
    return count


P = BoxModel(labels, size = n, replace = False)
X = RV(P, number_matches)
X.sim(10000)

<a id='plot'></a>
[Back to contents](#contents)

### Plot

Tabulate provides a tabular summary of the simulated values of a RV.  A plot summary can be obtained with `.plot()`.  The type of plot returned depends on the type of the random variable, discrete or continuous.

For *discrete* random variables, `.plot()` generally returns an "impulse" (or "spike") plot of the relative frequencies (by default) or frequencies (with `normalize = False`) for each of the possible values.

In [ ]:
P = BoxModel([0,1], size = 5)
X = RV(P, sum)
X.sim(10000).plot()

Symbulate allows a plot to be defined using either `x.plot()` or `plot(x)`.

In [ ]:
P = BoxModel([0,1], size = 5)
X = RV(P, sum)
plot(X.sim(10000), normalize = False)

For *continuous* random variables, `.plot()` returns a histogram, which provides the frequencies of simulated values falling in interval "bins".  By default, normalize = False and the vertical axis of the histogram is on the *density* scale, so that *areas* of the bars correspond to relative frequencies.  

In [ ]:
P = Normal(0,1)
X = RV(P)
X.sim(10000).plot()

The number of bins can be set manually with `bins=`.

In [ ]:
X.sim(10000).plot(bins = 30)

If many of the simulated values have small frequencies `.plot()` will return a histogram for both discrete and continuous random variables.

In [ ]:
P = BoxModel([0,1], size = 1000)
X = RV(P, sum)
X.sim(100).plot()

The type of plot, impulse versus histogram, can be overriden with the `type =` option, which can be set to either `type = 'impulse'` or `type = 'bar'` (the latter type for histograms).

In [ ]:
P = BoxModel([0,1], size = 1000)
X = RV(P, sum)
X.sim(100).plot(type = 'impulse', normalize = False)

It is not recommended to use `type = 'impulse'` for continuous random variables.

In [ ]:
P = Normal(0,1)
X = RV(P)
X.sim(100).plot(type = 'impulse')

Several distributions can be overlaid on the same plot.  The parameter `alpha`, which can take a value between 0 and 1, controls the level of transparency.

In [ ]:
for mu in [-2, 0, 2]:
    RV(Normal(mean = mu, sd = 1)).sim(10000).plot(alpha = 0.5)

When overlaying several discrete distributions in a plot, it is recommended to use the `jitter = True` option so that the bars do not coincide.

In [ ]:
RV(Binomial(10, 0.1)).sim(10000).plot(jitter = True)
RV(Poisson(1)).sim(10000).plot(jitter = True)

<a id='distribution'></a>
[Back to contents](#contents)

### Distribution of a random variable

Tabulate and plot provide summaries of the *distribution* of a random variable.  A distribution is a probability model on the real line which describes the possible values that the random variable can take and their relative likelihoods.   The distribution of a random variable is determined by (1) the underlying probability space on which the random variable is defined and (2) the function (mapping the probability space outcomes to real numbers) that defines the random variable.

It also common to specify the distribution of a random variable directly.  For example, a problem may  state "let $X$ have a Binomial(5, 0.5) distribution" without mention of the underlying probabilty space or random variable.  `RV` can also be used to define a random variable by specifying its distribution.  (Many [common probability distributions](common.html) are built into Symbulate.)

In [ ]:
X = RV(Binomial(n = 5, p = 0.5))
X.sim(10000).plot()

Note that specifying a random variable by specifying its distribution in this way has the effect of defining the probability space to be the distribution of the random variable and the function defined on this space to be the identity.  However, it is more appropriate to think of defining a random variable with the given distribution on an *unspecified probability space* through an *unspecified function*.

<a id='transform'></a>
[Back to contents](#contents)

### Functions of a random variable

Functions of random variables are also random variables.  If X is an RV, define a new RV $Y=g(X)$ using `X.apply(g)`.  The resulting $Y$ behaves like any other RV.

Note that for many math functions such as `exp` and `log` and `sin`, you can simply call `g(X)` rather than `X.apply(g)`.

*Example.* If $X$ has a normal distribution, then $e^X$ has a *lognormal* distribution.

In [ ]:
P = Normal(0,1)
X = RV(P)